In [1]:
# %load runNewFeaturesSA1.py
import ggcnn.experiment as experiment

import numpy as np
import sys
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.decomposition import PCA


def load_dataset():
    keys = []
    features = []
    labels = []

    with open('Data/2018-08-24-NSW-SA1Input-Normalised.csv', 'r') as file:
        for i, line in enumerate(file):
            if i == 0:  # Skip first line (header)
                continue
            s = line[:-1].split(',')  # Last value in line is \n
            keys.append(s[0])
            features.extend([float(v) for v in s[1:-1]])  # Last column is the outcome y
#             labels.append(np.floor(float(s[-1]) / 10).astype(int))
            labels.append(float(s[-1]))
        labels = np.array(labels)
        features = np.array(features).reshape((len(keys), -1))
    
    with open('Data/2018-08-25-NSW-NeighbourDistance.csv', 'r') as file:
        adj_mat = np.zeros((len(labels), 1, len(labels)))
        distance_mat = np.zeros((len(labels), len(labels)))
        for i, line in enumerate(file):
            if i == 0:  # Skip first line (header)
                continue
            s = line[:-1].split(',')
            a = keys.index(s[0])
            b = keys.index(s[1])
            adj_mat[a, 0, b] = 1
            adj_mat[b, 0, a] = 1
            distance_mat[a, b] = float(s[2])
            distance_mat[b, a] = float(s[2])

#     pca = PCA(n_components=0.7, svd_solver='full')
#     reduced_features = pca.fit_transform(features)
#     print(pca.explained_variance_ratio_)

    return features, adj_mat, labels

dataset = load_dataset()

# print(dataset[-2].shape)

class SA1Experiment():
    def __init__(self, neurons, blocks):
        self.blocks = blocks
        self.neurons = neurons
    
    def create_network(self, net, input):
        net.create_network(input)
#         net.make_adjacency_adjustment_layer()
        net.make_embedding_layer(self.neurons)
        net.make_dropout_layer()
        
        for _ in range(self.blocks):
            net.make_graphcnn_layer(self.neurons)
            net.make_dropout_layer()
            net.make_embedding_layer(self.neurons)
            net.make_dropout_layer()
        
        net.make_embedding_layer(self.neurons)
        net.make_embedding_layer(1, name='final', with_bn=False, with_act_func = False)


no_folds = 5
inst = KFold(n_splits = no_folds, shuffle=True, random_state=125)

l = 3
n = 128
i = 2

    
exp = experiment.GGCNNExperiment('2018-08-24-SA1', '2018-08-24-sa1', SA1Experiment(neurons = n, blocks = l))

exp.num_iterations = 2000
exp.optimizer = 'adam'
exp.loss_type = 'linear'

exp.debug = True  # Was True

exp.preprocess_data(dataset)

train_idx, test_idx = list(inst.split(np.arange(len(dataset[0]))))[i]
# test_idx, train_idx = list(inst.split(np.arange(len(dataset[-1]))))[i]  # Reversed to get more samples in the test set than the training set


exp.create_data(train_idx, test_idx)
exp.build_network()
results = exp.run()


/home/paperspace/anaconda3/envs/tf-gpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/paperspace/anaconda3/envs/tf-gpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


2018-08-28 23:54:30.183974 Creating training Tensorflow Tensors
2018-08-28 23:54:30.184924 Creating training network
2018-08-28 23:54:31.218286 Creating loss function and summaries
2018-08-28 23:54:31.277233 Training model "2018-08-24-sa1"!
2018-08-28 23:54:31.277716 Preparing training
2018-08-28 23:54:34.815709 Starting threads
2018-08-28 23:54:34.816347 Starting training. train_batch_size: 0 test_batch_size: 0
2018-08-28 23:54:35.778227 Test Step 0 Finished
2018-08-28 23:54:35.778920 Test Step 0 "min loss" =  6.8207516e+22
2018-08-28 23:54:35.779454 Test Step 0 "loss" =  6.8207516e+22
2018-08-28 23:54:38.345963 Training Step 0 Finished Timing (Training: 0.727133, Test: 0.272368) after 3.52888 seconds
2018-08-28 23:54:38.346512 Training Step 0 "min loss" =  2757.1636
2018-08-28 23:54:38.346858 Training Step 0 "loss" =  2757.1636
2018-08-28 23:54:38.618702 Test Step 5 Finished
2018-08-28 23:54:38.619232 Test Step 5 "min loss" =  2787.5217
2018-08-28 23:54:38.619602 Test Step 5 "loss" =

KeyboardInterrupt: 

In [ ]:
results.sum(axis = 0)

In [ ]:
dataset[1].sum(axis = 0)

In [ ]:
np.min(results - dataset[1])

In [ ]:
import matplotlib.pyplot as plt

plt.hist(results.ravel() - dataset[1].ravel(), bins = 50)

In [ ]:
###
train_df = pd.DataFrame(results[0])
test_df = pd.DataFrame(results[1])
test_df.set_index(test_df.index * exp.iterations_per_test, inplace = True)

In [ ]:
import matplotlib.pyplot as plt

train_df['loss'].loc[500:].plot()
test_df['loss'].loc[500:].plot()
train_df['min loss'].loc[500:].plot()
test_df['min loss'].loc[500:].plot()
print(test_df['min loss'].iloc[-1])

plt.show()

In [ ]:
import matplotlib.pyplot as plt

train_df['loss'].loc[500:].plot()
test_df['loss'].loc[500:].plot()
train_df['min loss'].loc[500:].plot()
test_df['min loss'].loc[500:].plot()
print(test_df['min loss'].iloc[-1])

plt.show()